# Introduzione

L'environment **anaconda** da utilizzare è **ddmcs**.  

# Importazione pacchetti

Importare i vari pacchetti necessari  per eseguire le azioni elencate nell'introduzione:
   - selenium (per navigare sulle pagine web)
   - request (scaricare dati da internet)
   - bs4 (beautiful soup, manipolare gli html)
   - os (navigare nei file del computer)
   - pandas (manipolazione/esportazione dataframes)
   - time (per i time sleep)
   - datetime (gestire i timestamp, ovvero le stringhe contenti date e orari)
   - tqdm (per le barre di progresso) 
   - re (lavorare con regex e le regular expressions)

In [ ]:
# Selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service # Selezionare il chrome driver
#-------##-------##-------##-------##-------##-------##-------##-------##-------##-------#
from bs4 import BeautifulSoup # Importazione pacchetti per gestire HTML
import os # Interagire con i file del pc 
from tqdm import tqdm as tqdm # Importazione pacchetti per barra di progresso
import time # Importazione pacchetti per utilizzare dei time.sleep()
import random # Importazione pacchetti per ottenere numeri casuali, utile per utilizzare time.sleep() randomici (meno identificabili)
#from datetime import datetime # Importazione pacchetti per lavorare con timestamps, orari e date

import json # Lavorare con file json

from pathlib import Path # Per identificare il path del file 'config'

# Identificare il file config.json 
start = Path.cwd()
current = start.resolve()
for parent in [current] + list(current.parents):
   json_path_config_path = parent / "config.json"
   if json_path_config_path.exists():
      break 

with open(json_path_config_path) as f:
   paths = json.load(f)

# Importare sys per indicare il path della cartella ove sono collocate le funzioni selfmade.
import sys
sys.path.append(f'{paths["path_progetto"]}{paths["path_funzioni_booking"]}')


# Importare le funzioni per lavorare su instagram con il selenium both
from funzioni_selenium_bot_booking import *


# Percorsi file 
Siccome questo bot leggerà e scriverà file localmente, stabilire quali sono i percorsi file di questi elementi.   

In [6]:
# Path per il driver di selenium
path_driver= paths["path_selenium_driver"]

# Directory principale dello scraper, ove vi sono gli script e gli html devono essere posizionati
path_main_directory = f"{paths['path_progetto']}{paths['path_scraper_booking']}"

Creare le directory ove mettere i file

In [7]:
# Andare alla directory principale specificata nella precedente linea di codice
os.chdir(path_main_directory)

# Se non c'è la directory html crearla. Viene inserita questa clausola per evitare che una directory esistente
# venga cancellata
if "html" not in os.listdir():
   os.mkdir("html")


# Impostare il selenium bot e accesso ad Instagram

In [ ]:
# La classe Service di Selenium viene utilizzata per gestire e configurare il driver del browser
service = Service(executable_path=path_driver)

# Configurare il driver di Chrome 
options = webdriver.ChromeOptions()

# Questa impostazionie permette di non caricare le immagini, rende lo scraping più veloce.
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

# Definire il driver: utilizzare il driver scelto (service) e passargli le impostazioni scelte (options)
driver = webdriver.Chrome(service=service, options=options)

# Impostare le dimensioni della finestra google sulla quale il bot selenium opera.
driver.maximize_window()            # dimensioni massime per la finestra
#driver.set_window_size(600, 800)   # dimensioni customizzate per la finestra

# Istruzioni per il selenium bot 

Il sito web **booking** non suddivide i risultati in diverse pagine, si possono caricare risultati aggiuntivi continuando a scrollare la pagina.

Questo **bot** continua l'estrazione dati finchè non si arriva fondo della pagina, ci si assicura questo tramite l'uso di un **while loop**.

In [ ]:
# Specificare il link della pagina dalla quale estrarre i dati.
link = """https://www.booking.com/searchresults.it.html?ss=Roma%2C+Lazio%2C+Italia&efdco=1&label=it-it-booking-desktop-VRZD0IC5lt9Ulq*ajTZ_bgS652829000338%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9198605%3Ali%3Adec%3Adm&aid=2311236&lang=it&sb=1&src_elem=sb&src=index&dest_id=-126693&dest_type=city&place_id=city%2F-126693&ac_position=0&ac_click_type=b&ac_langcode=it&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=c9c296d9b1a203be&ac_meta=GhBjOWMyOTZkOWIxYTIwM2JlIAAoATICaXQ6AnJvQABKAFAA&checkin=2025-08-01&checkout=2025-08-02&group_adults=2&no_rooms=1&group_children=0"""

# Andare alla pagina assegnata alla variabile 'link' 
driver.get(link) 

# Aspettare 10 secondi per far caricare la pagina
time.sleep(10)

# Accettare i cookie
click_cookie_button(driver)


# Parametri per scrollare la pagina
scroll_step = 200 # Di quanto scrollare la pagina verso il basso ogni iterazione
current_position = 0 # Posizioni di inizio
scroll = 0 # Numero di scroll
last_height = driver.execute_script("return document.body.scrollHeight") # Altezza della pagina, sarà utile per capire se il fondo della pagina è stato raggiunto. Quando last_height = current_position il fondo della pagina è raggiunto.

# Loop per l'estrazione dell'html
while (current_position < last_height):
   # Prendere l'html della pagina 
   html=driver.page_source 
   

   # Capire se compare il carica altri button e provare a cliccarlo
   html_bs4 = BeautifulSoup(html,'html.parser')
   if html_bs4.find("button",{"class":"de576f5064 b46cd7aad7 d0a01e3d83 dda427e6b5 bbf83acb81 a0ddd706cc"}) != None:
      click_caricaaltri_button(driver)


   # Salvare l'html in un file txt
   with open(f"{path_main_directory}/html/html_{scroll}.txt","w+") as f:
      f.write(str(html))    

   # Scrollare la pagina
   driver.execute_script(f"window.scrollBy(0, {scroll_step});")
   
   # Tempo di attesa randomico, meno riconoscibile
   tempo_attesa = random.randint(1,3)
   time.sleep(tempo_attesa)
      
   # Aggiornare i parametri di scroll
   current_position += scroll_step    # Aggiornare la current_position
   last_height = driver.execute_script("return document.body.scrollHeight") # Aggiornare la last_height
   scroll += 1 # Aggiornare il numero di scroll 




# Salvare l'html in un file txt
with open(f"{path_main_directory}/html/html_final.txt","w+") as f:
   f.write(str(html)) 